In [ ]:

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import pandas as pd
from numpy.random import seed
%matplotlib inline
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import os
import gc
import re
import cv2
import gc
import time
seed(1)

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

# create X, y

In [ ]:

    
def create_npy(img_folder):
    Dataset_energy=pd.read_table('data/Indices_MolPlan.dat',sep=' ',header=None,names=['mol_index','energy'])
    img_data_array=[]
    energy=[]
    t0=time.time()
    i=0
    for file in os.listdir(img_folder):

        image_path= os.path.join(img_folder,  file)
        image= cv2.imread(image_path)
        image=cv2.resize(image, (64,64),interpolation = cv2.INTER_AREA)
        image=np.array(image)
        image = (image.astype('float32')/255).round(1)
        img_data_array.append(image)
        index=int(re.split('[_,.]',file)[1])
        E=Dataset_energy[Dataset_energy['mol_index']==index].iloc[0,1]
        energy.append(E)
        i+=1
    energy=np.array(energy).reshape((4237,1))
    img_data_array=np.array(img_data_array)
    return img_data_array,energy

    






# Data augmentation by rotation of molecules

In [ ]:
def rotate(image,angle,center=(32,32), scale=1):
        M = cv2.getRotationMatrix2D(center, angle, scale)
        (h, w) = image.shape[:2]
        rotated = cv2.warpAffine(image, M, (w, h))
        return rotated

img_augmented=[]
for i in range(0,len(img_data)):
    image=img_data[i,:,:,:]
    img_augmented.append(image)
    for j in range(1,12):
        image_rt=rotate(image,30*j)
        img_augmented.append(np.array(image_rt))
img_augmented=np.array(img_augmented)

# ResNet50 model

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s),name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides = (1,1), padding='same',name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides = (1,1),name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s),name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    return X
    
    
def identity_block(X, f, filters, stage, block):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    
    return X
    
def ChemModel():

    # Define the input as a tensor with shape input_shape
    X_input = Input([64,64,3])


    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4 ,block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4 ,block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2),name='avg_pool')(X)


    # output layer
    X = Flatten()(X)
    X = Dense(256, activation='relu')(X)
    X = Dense(128, activation='relu')(X)
    X = Dense(1)(X)


    model = Model(inputs = X_input, outputs = X, name='ChemModel')

    return model

In [ ]:

ChemModel=ChemModel()
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

epochs=100
ChemModel.compile(optimizer='adam',loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
history= ChemModel.fit(img_data,energy, batch_size=32,epochs=epochs, callbacks=[callback],validation_split=0.2)
   
  

# Evaluation

In [ ]:
fig,ax=plt.subplots()
ax.plot(history.history['root_mean_squared_error'])
ax.plot(history.history['val_root_mean_squared_error'])
ax.set_title('Data_augmented')
ax.set_yscale("log")
ax.set_ylabel('RMSE')
ax.set_xlabel('Epoch')

plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.savefig('loss.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt


loss = history.history['root_mean_squared_error']
val_loss = history.history['val_root_mean_squared_error']

rmse_train = np.sqrt(loss[-1] * (energy_max - energy_min)**2)
rmse_val = np.sqrt(val_loss[-1] * (energy_max - energy_min)**2)

epochs = range(100)

plt.plot(epochs, loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'g-', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.legend()
plt.show()